# Embedding Agent

What if I trained an agent on word embeddings through reinforcement learning, to predict the results of *semantic equations* à la "king - man = queen"?

Can I study the world model it builds, and thus deduce things about the world models that could be built from the semantic embeddings?
Can I apply IRL methodology to study the reward structure?
Can I use the agents knowledge of semantics to have it act in a (ad hoc and virtual) world?

(To download the glove embeddings, go here: https://nlp.stanford.edu/projects/glove/)

## Imports

In [1]:
import os
import random

In [2]:
from gensim.models import KeyedVectors

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

import spacy

## Embedding space investigations

In [3]:
nlp = spacy.load("en_core_web_md")

glove_dir = "glove6B"
glove_file = os.path.join(glove_dir, "glove.6B.100d.txt")

In [4]:
embeddings_dict = {}
with open(glove_file, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        
        token = values[0]
        embeddings = list(map(float, values[1:]))
        
        embeddings_dict[token] = embeddings

In [6]:
# get parameter count and embedding dimension
_, pc, ed, _ = glove_file.split(".")
path_w2v = os.path.abspath(f'glove6B/glove.{pc}.{ed}.w2v.txt')

glove = datapath(glove_file)
w2v = get_tmpfile(path_w2v)

model = KeyedVectors.load_word2vec_format(w2v, binary=False)

/var/folders/m2/j205f6kd4lsc468zllr25wbw0000gn/T/ipykernel_5581/4011947216.py:8: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, w2v)


In [7]:
nouns = []
for token in list(embeddings_dict.keys())[:3_000]:
    doc = nlp(token)
    for token in doc:
        if token.pos_ == 'NOUN':
            nouns.append(token.text)

In [23]:
noun_set = set(nouns)
# similarity threshold (controls the quality of the data)
threshold = 0.7

In [24]:
filtered = []
for _ in range(5_000):
    word1 = random.choice(list(noun_set))
    word2 = random.choice(list(noun_set - {word1}))

    result = model[word1] - model[word2]

    similar_words = model.similar_by_vector(result, topn=10)
    filtered_similar_words = [(word1, word2, word, score) for word, score in similar_words if score > threshold]

    if filtered_similar_words:
        filtered.append(filtered_similar_words)

In [25]:
filtered

[[('violence', 'a.', 'violence', 0.725869357585907)],
 [('index', 'choice', 'index', 0.8321520686149597),
  ('index', 'choice', 'composite', 0.7088848948478699),
  ('index', 'choice', 'indexes', 0.70453941822052)],
 [('fighters', 'reasons', 'fighters', 0.7353002429008484)],
 [('elections', 'medicine', 'elections', 0.7158864140510559)],
 [('river', 'dealers', 'river', 0.7598223090171814)],
 [('income', 'spot', 'income', 0.7214859127998352)],
 [('sentence', 'personnel', 'sentence', 0.7246450185775757)],
 [('bangladesh', 'brothers', 'bangladesh', 0.7059545516967773)],
 [('communist', 'bell', 'communist', 0.7608583569526672)],
 [('test', 'commerce', 'test', 0.718548059463501)],
 [('municipality', 'headquarters', 'municipality', 0.7010007500648499)],
 [('=', 'arts', '=', 0.7116696834564209)],
 [('news', 'forms', 'news', 0.7362059950828552)],
 [('banks', 'author', 'banks', 0.713893711566925)],
 [('analyst', 'fort', 'analyst', 0.7079832553863525)],
 [('spokesman', 'elements', 'spokesman', 0.7

This is much worse than I expected...